### Training auto-encoder with distance lose
#### Written by: @leopauly | cnlp@leeds.ac.uk

In [4]:
## Imports
from keras.models import Sequential
import random
import numpy as np
from PIL import Image
from os import listdir
from scipy.ndimage import imread
import keras
import tensorflow as tf
from tensorflow.python.client import device_lib
from keras import backend as K
import datetime
import time
import os 
from datetime import timedelta
## Custom scripts
import lscript as lsp
import modelling as md
print('Loaded libraries...!!',flush=True)


Loaded libraries...!!


In [5]:
height=112 
width=112 
channel=3
cluster_length=16
nb_classes=101
lr_rate=.001
next_batch_start=0
batch_size=16
batch_size_test=16
total_train_videos=9991
memory_batch_size_train=9991
memory_batch_size_test=3329
iterations= 30 
custom_global_step=0
LOG_DIR='/nobackup/leopauly/logdirk80_1_extnd'
saved_path='/nobackup/leopauly/S2l/'
best_validation_accuracy=0.0
best_iteration=0
print('Finished defining variables..!!',flush=True)

Finished defining variables..!!


In [ ]:
#------------------------------------------------------------------------------------------------------------------------#

## Training & testing
def testing(iterations,loops):
    test_score=0
    for j in range(int(memory_batch_size_test/batch_size_test)-1):
        test_score_ = sess.run([accuracy], feed_dict={x_image:test_images[(batch_size_test*j):(batch_size_test*(j+1))],y_true_cls:test_labels_cls[(batch_size_test*j):(batch_size_test*(j+1))],K.learning_phase(): 0 })
        test_score=test_score+sum(test_score_)
    
      
    print('Test score after iteration:',iterations,',loop:',loops,'is:',test_score/(j+1),flush=True)
    validation_accuracy= (test_score/(j+1))*100
    return validation_accuracy


#------------------------------------------------------------------------------------------------------------------------#

In [ ]:
## Start the session with logging placement.
init_op = tf.global_variables_initializer()
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
sess.run(init_op)

## Restore model weights from previously saved model 
saver = tf.train.Saver()
saver.restore(sess, os.path.join(saved_path,'activity_model.ckpt-67'))
print("Model restored from file: %s" % saved_path,flush=True)
print ('started the session...!!',flush=True)

## Loading Training data
start_time = time.time()
train_images, train_labels_cls, next_batch_start, _ = ucf.read_vid_and_label('./UCF101_data_preparation/train.list',memory_batch_size_train,-1,cluster_length,112,normalisation=False)
end_time = time.time()
time_dif = end_time - start_time
print("Time usage for loading training dataset: " + str(timedelta(seconds=int(round(time_dif)))),flush=True)
train_labels=keras.utils.to_categorical(train_labels_cls, num_classes=nb_classes)

## Loading Testing data
test_images, test_labels_cls, next_batch_start, _ = ucf.read_vid_and_label('./UCF101_data_preparation/test.list',memory_batch_size_test,-1,16,112,normalisation=False)
test_labels=keras.utils.to_categorical(test_labels_cls, num_classes=nb_classes)
print('testing data loaded',flush=True)


#------------------------------------------------------------------------------------------------------------------------#

In [ ]:

## Training
for i in range(0,(iterations*10)):
    print('started iteration:',i,flush=True)
    
    for j in range (int(memory_batch_size_train/batch_size)-1):    
        print ('This is epoch:',j,'going to be trained',flush=True)
        output_value = sess.run([optimizer], feed_dict={x_image:train_images[(batch_size*j):(batch_size*(j+1))],y_true:train_labels[(batch_size*j):(batch_size*(j+1))],K.learning_phase(): 1 })   
        print ('This is epoch:',j,'trained',flush=True)
            
    validation_accuracy=testing(i,j)
    print(' validation_accuracy in iteration:',i,'is:', validation_accuracy)
    
    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        best_iteration=i
        saver.save(sess, os.path.join(LOG_DIR, "activity_model.ckpt"), global_step=i)
        custom_global_step=custom_global_step+1   
        print('Model saved after iteration:',best_iteration,flush=True)

print('best_validation_accuracy:',best_validation_accuracy)      
print('best_iteration:',best_iteration)

sess.close()


#------------------------------------------------------------------------------------------------------------------------#